In [2]:
import numpy as np

class BayesianNetwork:


    def __init__(self, variables, cpt_dict):

        self.variables = variables
        self.cpt_dict = cpt_dict

    def joint_probability(self, assignment):

        probability = 1.0
        for variable in self.variables:
            parents = self.cpt_dict[variable]['parents']
            parent_values = tuple([assignment[parent] for parent in parents]) if parents else ()
            probability *= self.cpt_dict[variable]['cpt'][parent_values][assignment[variable]]
        return probability

    def variable_elimination(self, query_variables, evidence):


        all_possible_assignments = self.generate_all_assignments()

        posterior_distribution = {}
        total_probability = 0.0

        for assignment in all_possible_assignments:
          if all(assignment[var] == evidence[var] for var in evidence):
            joint_prob = self.joint_probability(assignment)

            query_assignment = tuple([assignment[var] for var in query_variables])

            if query_assignment not in posterior_distribution:
              posterior_distribution[query_assignment] = 0.0

            posterior_distribution[query_assignment] += joint_prob
            total_probability += joint_prob

        for assignment in posterior_distribution:
          posterior_distribution[assignment] = posterior_distribution[assignment] / total_probability

        return posterior_distribution


    def generate_all_assignments(self):

      all_assignments = []
      def generate_recursive(current_assignment, remaining_variables):
          if not remaining_variables:
              all_assignments.append(current_assignment)
              return

          variable = remaining_variables[0]
          for value in [0, 1]:
              new_assignment = current_assignment.copy()
              new_assignment[variable] = value
              generate_recursive(new_assignment, remaining_variables[1:])

      generate_recursive({}, self.variables)
      return all_assignments

# Example Usage:
variables = ['A', 'B', 'C']
cpt_dict = {
    'A': {'parents': [], 'cpt': {(): {0: 0.7, 1: 0.3}}},
    'B': {'parents': ['A'], 'cpt': {(0,): {0: 0.9, 1: 0.1}, (1,): {0: 0.2, 1: 0.8}}},
    'C': {'parents': ['B'], 'cpt': {(0,): {0: 0.8, 1: 0.2}, (1,): {0: 0.5, 1: 0.5}}},
}


# Create a Bayesian Network
bn = BayesianNetwork(variables, cpt_dict)

# Example query: P(A | B=1)
query_variables = ['A']
evidence = {'B': 1}

posterior_distribution = bn.variable_elimination(query_variables, evidence)
print("Posterior Distribution for P(A|B=1):", posterior_distribution)


Posterior Distribution for P(A|B=1): {(0,): 0.2258064516129032, (1,): 0.7741935483870968}
